In [14]:
#Testing what will happen if we don't reshape our 1D flattened images to 3D original shape

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import tensorflow as tf
import keras

from tensorflow.keras import datasets, layers, models
        
#train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
#test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

EPOCHS = 10
BATCH_SIZE = 32
VALIDATION_SIZE = 0.2

In [15]:
X_train = train_df.drop('label', axis=1).to_numpy()
y_train = train_df['label']

X_train_reshaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train_normalized = X_train_reshaped / 255.0

In [17]:
print("Shape of X_train_normalized: ", X_train_normalized.shape)
print("Shape of y_train: ", y_train.shape)

Shape of X_train_normalized:  (42000, 28, 28, 1)
Shape of y_train:  (42000,)


In [18]:
# Define the CNN model
model = models.Sequential()


# Convolutional layers
model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten and fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

c:\Users\vlns\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [19]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [20]:
model.fit(X_train_normalized, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SIZE)

Epoch 1/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8623 - loss: 0.4372 - val_accuracy: 0.9787 - val_loss: 0.0683
Epoch 2/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9783 - loss: 0.0699 - val_accuracy: 0.9856 - val_loss: 0.0472
Epoch 3/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9880 - loss: 0.0377 - val_accuracy: 0.9885 - val_loss: 0.0401
Epoch 4/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9911 - loss: 0.0271 - val_accuracy: 0.9887 - val_loss: 0.0432
Epoch 5/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9938 - loss: 0.0206 - val_accuracy: 0.9885 - val_loss: 0.0401
Epoch 6/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9948 - loss: 0.0158 - val_accuracy: 0.9900 - val_loss: 0.0392
Epoch 7/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9959 - loss: 0.0114 - val_accuracy: 0.9788 - val_loss: 0.1062
Epoch 8/10
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9956 - loss: 0.0134 - 

In [21]:
X_test = test_df.to_numpy().reshape(-1, 28, 28, 1)
X_test_normalized = X_test / 255.0
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [22]:
predicted_df = pd.DataFrame(predicted_labels)
predicted_df.insert(0, 'ImageId', [i for i in range(1, len(predicted_df) + 1)])
predicted_df.columns = ['ImageId', 'Label']
predicted_df.to_csv('data/reshaped.csv', index=False)